In [3]:
from clickhouse_driver import Client
import io
import pandas as pd
import numpy as np
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
import psycopg2
import requests
import json
import time
from requests_futures.sessions import FuturesSession
from backend_logic import *
session = FuturesSession()

offerid = '5d5d6def7e76801af8637446'
maxRivalsCount = 12
sqiDiffCoef = 100
maxPos = 10
minCountInSerm = 6

offer = getOffer(offerid)

# Получить ключи из КП
keywords = []
keywordTexts = []
for keyword in offer['KeywordsToSave']:
    text = keyword[1]['Text']
    if "'" in text:
        text = text.replace("'", "''")
    keywords.append({
        'id': keyword[0],
        'text': text
    })
    keywordTexts.append(text)
keywordsStr = u"'" + u"', '".join(keywordTexts) + u"'"

# Получить дату создания из КП
startDate = offer['CreationTime'] - timedelta(days=5)
startDate = startDate.strftime('%Y-%m-%d')

# Получить домен из КП
offerDomain = offer['Domain'].replace('www.', '')

# Получить регионы из КП
offerRegions = getOfferRegions(offer)

# Получить id регионов из КП для Яндекса и Гугла
regions = getRegionIds(offerRegions)
regionsStr = u"'" + u"', '".join([x['yaId'] for x in regions])\
        + u"', '" + u"', '".join([x['gId'] for x in regions]) + u"'"

# Получить конкурентов из базы
query =f"""
    select domain, count(*)
    from secache3
    where
        keyword in ({keywordsStr})
        and date >= '{startDate}'
        and regionId in ({regionsStr})
        and position <= {maxPos}
        and isMobile = 0
    group by domain
    having count(*) >= {minCountInSerm}
    order by count(*) desc
    limit {maxRivalsCount}
"""

client = Client(host = '192.168.10.101', user = 'read_only', password = 'User_ro', settings={'max_query_size':500000})
data = client.execute(query)
client.disconnect()

# Получить популярные домены из файла конфига
excluded_domains = []
with io.open("./popular-domains.txt", mode="r", encoding="utf-8") as f:
    for line in f:
        excluded_domains.append(str.strip(line))
excluded_domains.append(offerDomain)

# Заполнить данные по доменам
domains = []
ourSqi = getSqi(offerDomain)
hasOfferDomain = False
for domain in data:
    domainSqi = getSqi(domain[0])
    isExcluded = False
    for ex in excluded_domains:
        if ex == domain[0] or '.' + ex in domain[0]:
            isExcluded = True
    domains.append({
        'domain': domain[0],
        'countInSerm': domain[1],
        'isExcluded': isExcluded,
        'sqi': getSqiPromise(domain[0]),
        'isOfferDomain': domain[0] == offerDomain
    })
    if domain[0] == offerDomain:
        hasOfferDomain = True
if not hasOfferDomain:
    domains.append({
        'domain': offerDomain,
        'countInSerm': 0,
        'isExcluded': False,
        'sqiDiff': 1,
        'sqi': ourSqi,
        'isOfferDomain': True
    })
for domain in domains:
    try:
        sqi = getSqiResult(domain['sqi'], domain['domain'])
        domain['sqi'] = sqi,
        domain['sqiDiff'] = sqi / ourSqi
        if(domain['sqiDiff']) > sqiDiffCoef:
            domain['isExcluded'] = True
    except:
        domain['sqi'] = 0
        domain['sqiDiff'] = 0

In [2]:
keywordsStr

"'купить крем vichy', 'vichy купить', 'купить крем виши', 'vichy купить в москве', 'vichy laboratoires цена', 'крем виши цена', 'купить виши со скидкой', 'крем виши ночной цена', 'купить ампулы виши', 'крем vichy', 'крем виши', 'bioderma sebium гель', 'биодерма себиум гель', 'биодерма умывалка', 'биодерма для умывания', 'биодерма себиум мусс', 'bioderma sebium мусс', 'биодерма себиум мусс очищающий', 'биодерма себиум гель мусс', 'биодерма очищающий мусс', 'биодерма очищающий гель', 'крем la roche posay effaclar', 'крем ля рош позе эфаклар дуо', 'крем effaclar duo', 'крем эфаклар дуо плюс', 'крем la roche posay effaclar duo', 'крем эфаклар дуо', 'крем для лица эфаклар дуо', 'la roche posay корректирующий крем effaclar duo', 'крем ля рош дуо', 'la roche posay effaclar duo купить', 'биодерма себиум цена', 'биодерма себиум гель цена', 'биодерма себиум гель для умывания цена', 'bioderma sebium купить', 'биодерма себиум купить', 'биодерма себиум купить в москве', 'bioderma sebium купить в мо

In [ ]:
query = """select domain, count(*)
    from secache3
    where
        keyword in ('купить крем vichy', 'vichy купить', 'купить крем виши', 'vichy купить в москве', 'vichy laboratoires цена', 'крем виши цена', 'купить виши со скидкой', 'крем виши ночной цена', 'купить ампулы виши', 'крем vichy', 'крем виши', 'bioderma sebium гель', 'биодерма себиум гель', 'биодерма умывалка', 'биодерма для умывания', 'биодерма себиум мусс', 'bioderma sebium мусс', 'биодерма себиум мусс очищающий', 'биодерма себиум гель мусс', 'биодерма очищающий мусс', 'биодерма очищающий гель', 'крем la roche posay effaclar', 'крем ля рош позе эфаклар дуо', 'крем effaclar duo', 'крем эфаклар дуо плюс', 'крем la roche posay effaclar duo', 'крем эфаклар дуо', 'крем для лица эфаклар дуо', 'la roche posay корректирующий крем effaclar duo', 'крем ля рош дуо', 'la roche posay effaclar duo купить', 'биодерма себиум цена', 'биодерма себиум гель цена', 'биодерма себиум гель для умывания цена', 'bioderma sebium купить', 'биодерма себиум купить', 'биодерма себиум купить в москве', 'bioderma sebium купить в москве', 'крем bioderma sebium', 'крем себиум', 'биодерма себиум мыло купить', 'крем cicaplast', 'крем ля рош цикапласт', 'крем ля рош позе цикапласт', 'крем цикапласт la roche posay', 'цикапласт бальзам b5', 'крем cicaplast baume b5', 'крем la roche posay cicaplast baume b5', 'цикапласт бауме б5', 'цикапласт бальзам', 'крем weleda', 'веледа крем', 'веледа купить', 'weleda купить в москве', 'купить веледа в москве', 'купить крем веледа', 'крем weleda купить', 'веледа мазь', 'крем для тела weleda', 'купить крем la roche posay', 'купить крем ля рош позе', 'la roche posay цена', 'la roche posay купить в москве', 'крем la roche posay', 'французская косметика ля рош позе', 'французская косметика ля рош', 'крем для тела la roche posay', 'биодерма себиум увлажняющий крем', 'bioderma sebium гидра крем', 'крем bioderma sebium hydra', 'биодерма себиум гидра', 'биодерма sebium hydra', 'биодерма себиум гидра крем увлажняющий', 'биодерма себиум гидра крем', 'bioderma sebium hydra купить', 'купить крем биодерма сенсибио', 'bioderma sensibio купить', 'bioderma sensibio цена', 'bioderma sensibio купить в москве', 'биодерма сенсибио купить', 'биодерма сенсибио цена', 'биодерма сенсибио ar крем купить', 'bioderma sebium akn купить', 'биодерма себиум акн купить', 'биодерма себиум акн', 'биодерма себиум akn', 'биодерма себиум себиум akn крем', 'биодерма акн', 'биодерма себиум акн эмульсия', 'bioderma atoderm купить', 'биодерма атодерм крем купить', 'биодерма атодерм купить', 'bioderma atoderm creme купить', 'биодерма атодерм цена', 'bioderma atoderm крем', 'биодерма атодерм крем', 'авен ксеракалм', 'авен ксеракалм крем', 'ксерокалм ад', 'авен ксеракалм ад', 'ксерокалм цена крем', 'avene xeracalm крем', 'биодерма сенсибио для глаз', 'bioderma крем для глаз', 'биодерма крем для глаз', 'биодерма крем для век', 'крем вокруг глаз биодерма', 'крем для кожи вокруг глаз биодерма', 'крем капилляропротектор для кожи склонной к куперозу', 'крем капилляропротектор кора', 'крем кора от купероза', 'крем капилляропротектор', 'крем кора от купероза цена', 'крем кора капилляропротектор купить', 'холи ленд супер лосьон купить', 'holy land super lotion купить в москве', 'holy land super lotion купить', 'holy land super lotion 250 мл купить', 'холи ленд супер лосьон', 'термальная вода uriage купить', 'термальная вода урьяж купить', 'термальная вода урьяж цена', 'термальная вода урьяж', 'термальная вода uriage', 'крем против растяжек при беременности', 'крем против растяжек для беременных', 'крем от растяжек для беременных', 'крема от растяжек при беременности', 'крем для тела для беременных', 'holika holika aloe купить', 'алоэ холика холика купить', 'holika holika aloe 99 купить', 'holika holika универсальный гель 99 алоэ вера', 'алоэ вера холика холика', 'авен матирующий лосьон', 'avene матирующий лосьон', 'авен клинанс лосьон очищающий матирующий', 'avene cleanance лосьон очищающий матирующий', 'клинанс лосьон', 'organic shop скраб', 'скраб органик шоп', 'скраб для тела organic shop', 'скраб для тела органик шоп', 'скраб для тела organic shop бразильский кофе', 'урьяж от купероза', 'крем урьяж розельян от купероза', 'крем урьяж розельян', 'крем uriage roseliane', 'крем розельян', 'holika holika купить', 'холика холика купить', 'holika holika москва', 'holika косметика', 'крем holika holika', 'крем для глаз ля рош позе', 'la roche posay крем для глаз', 'крем для глаз ля рош', 'крем вокруг глаз la roche posay', 'крем для век ля рош позе', 'купить крем либридерм', 'librederm цена', 'крем либридерм цена', 'заказать косметику либридерм', 'libre derm косметика', 'avene клинанс', 'авен клинанс очищающий гель', 'авен гель для умывания для проблемной кожи', 'avene для умывания', 'avene очищающий гель', 'крем с эстрогеном для лица', 'крем с эстрогенами', 'крем с эстрогенами для женщин', 'крем с фитоэстрогенами', 'крем с фитоэстрогенами для лица', 'domix green professional жидкое лезвие', 'la roche posay lipikar цена', 'la roche posay lipikar ap купить', 'крем липикар ап', 'крем lipikar baume ap', 'la roche posay липикар ап', 'крем для век avene', 'авен крем для глаз', 'avene крем для глаз', 'крем вокруг глаз авене', 'avene для глаз', 'крем для сжигания жира', 'жиросжигающий крем для живота', 'крем для сжигания жира на животе', 'жиросжигающий крем для лица', 'черная маска от черных точек купить москва', 'черная маска от черных точек купить', 'заказать черную маску для лица', 'черная маска для лица цена', 'биодерма цикабио крем', 'биодерма cicabio creme', 'биодерма цикабио', 'биодерма цикабио купить', 'vichy liftactiv yeux цена', 'vichy liftactiv supreme купить', 'крем виши лифтактив', 'крем виши дневной лифтактив', 'купить крем с гиалуроновой кислотой', 'купить крем для лица с гиалуроновой кислотой', 'купить крем гиалурон', 'крем для лица с гиалуроновой кислотой', 'биодерма концентрат для сужения пор', 'биодерма для сужения пор', 'bioderma сужение пор', 'bioderma pore refiner купить', 'крем для лица янсен', 'крем для лица janssen cosmetics', 'крем янссен', 'крем янсен', 'биодерма себиум глобаль', 'биодерма себиум глобаль крем', 'биодерма себиум глобал', 'биодерма глобал крем', 'биодерма сенсибио лайт крем', 'биодерма лайт крем', 'биодерма сенсибио лайт', 'крем bioderma sensibio light', 'avene питательный компенсирующий крем', 'авен питательный компенсирующий крем', 'авен компенсирующий крем', 'avene питательный крем', 'vichy крем для глаз', 'крем вокруг глаз vichy', 'vichy для кожи вокруг глаз', 'крем для глаз виши', 'крем антистресс кора цена', 'крем антистресс кора купить', 'крем антистресс кора', 'крем кора антистресс состав', 'крем биодерма сенсибио', 'крем bioderma sensibio', 'биодерма sensibio', 'биодерма для чувствительной кожи', 'биодерма цена', 'биодерма заказать', 'крем uriage', 'крем урьяж', 'uriage eau thermale купить', 'uriage isodense купить', 'крем холи ленд', 'крем для лица холи ленд', 'крем holy land', 'holy land крем для лица', 'биодерма гидрабио купить', 'bioderma сыворотка', 'биодерма гидрабио сыворотка', 'биодерма гидрабио серум', 'израильская косметика холи ленд', 'израильская косметика hl', 'профессиональная косметика холи ленд', 'израильская косметика holy land', 'weleda для беременных', 'веледа от растяжек для беременных', 'веледа крем от растяжек для беременных', 'weleda от растяжек при беременности', 'авен тоник', 'авен тоник для лица', 'авен мягкий тонизирующий лосьон', 'авен мягкий лосьон', 'биодерма сенсибио форте крем', 'крем сенсибио форте', 'биодерма сенсибио форте', 'фотодерм ar крем spf 50', 'биодерма фотодерм ar крем spf 50', 'биодерма фотодерм ar крем spf 50 купить', 'авен пенка для умывания', 'avene пенка для умывания', 'avene мусс', 'elizavecca купить', 'elizavecca milky piggy купить в москве', 'купить косметику elizavecca', 'крема для жирной кожи лица список лучших', 'крем для жирной кожи лица', 'крем против жирной кожи', 'мицеллярная вода биодерма цена', 'bioderma мицеллярная вода купить', 'мицеллярная вода биодерма купить москва', 'ciracle pore control купить', 'ciracle blackhead off sheet купить', 'ciracle pore control blackhead off sheet купить', 'крем для тела natura siberica', 'крем для тела натура сиберика', 'крем для тела сиберика', 'la roche bb крем', 'вв крем ла рош позе', 'бб крем ля рош позе', 'купить гель алоэ вера для лица', 'алоэ для лица купить', 'купить гель алоэ вера в москве', 'авен увлажняющий крем', 'avene увлажняющий крем', 'крем авен гидранс', 'крем dr sea', 'израильская косметика dr sea', 'dr sea крем для лица', 'тоник для проблемной кожи', 'антибактериальный тоник для лица', 'тоник для лица с цинком', 'крем noreva', 'крем для лица noreva', 'noreva тональный крем купить', 'крем для упругости кожи тела', 'крем для подтяжки кожи', 'крем лифтинг для тела', 'крем для лица с церамидами', 'крем с церамидами', 'крем с керамидами для лица', 'крем для лица антивозрастной', 'крем антивозрастной дневной для лица', 'крем для лица омолаживающий', 'avene мицеллярная вода', 'мицеллярная вода авен', 'avene мицеллярная вода купить', 'авен сикальфат', 'avene cicalfate купить', 'авен крем восстанавливающий', 'патчи с плацентой', 'berrisom патчи', 'berrisom placenta патчи', 'скраб облепиховый натура сиберика', 'скраб облепиха натура сиберика', 'скраб для кожи головы натура сиберика', 'аравия карбокситерапия купить', 'карбокситерапия aravia купить', 'карбокситерапия для лица аравия', 'крем для сухой кожи лица', 'крем для очень сухой кожи лица', 'крем от сухости кожи', 'биодерма крем от купероза', 'биодерма от купероза', 'крем биодерма ар', 'холи ленд отбеливающий крем', 'holy land отбеливающий крем', 'holy land whitening cream купить', 'израильская профессиональная косметика', 'израильская косметика для лица профессиональная', 'израильская лечебная косметика', 'термальная вода авен купить', 'термальная вода авен', 'авен термальная вода 150 мл цена', 'авен трикзера', 'avene трикзера', 'авен трикзера купить', 'la roche posay effaclar гель купить', 'la roche posay effaclar gel купить', 'la roche posay effaclar гель', 'крем от купероза', 'крем против купероза', 'крем для кожи с куперозом', 'биодерма сенсибио ds', 'биодерма сенсибио дс', 'биодерма сенсибио ds крем', 'avene термальная вода 300 мл купить', 'авен термальная вода 300 мл купить', 'avene термальная вода 300 мл цена', 'урьяж ксемоз купить', 'крем ксемоз', 'крем uriage xemose', 'крем для осветления кожи', 'крем для отбеливания кожи лица', 'крем для отбеливания кожи', 'ампульные сыворотки для лица', 'антивозрастная сыворотка для лица', 'крем теана', 'teana косметика', 'крем коэнзим q10', 'крем коэнзим', 'кислородный гель с гиалуроновой кислотой', 'кислородный гель с гиалуроновой кислотой beauty style', 'librederm гиалуроновый филлер', 'librederm гиалуроновый 3d филлер', 'holy land алоэ гель', 'алоэ гель холи ленд', 'librederm пилинг скатка с ромашкой для лица', 'librederm пилинг скатка', 'крем кора для возрастной кожи 55', 'крем кора антивозрастной 50', 'guam скраб', 'guam скраб для тела', 'крем с азуленом', 'крем с азуленом для лица', 'урьяж очищающий пенящийся крем', 'uriage очищающий пенящийся крем', 'avene матирующий флюид', 'avene увлажняющий матирующий флюид', 'крем natura siberica для чувствительной кожи', 'крем натура сиберика для чувствительной кожи', 'la roche posay effaclar лосьон', 'la roche posay effaclar тоник', 'ля рош позе от пигментных пятен', 'la roche posay от пигментных пятен', 'крем детокс', 'янсен детокс крем', 'крем mustela для лица', 'крем мустела для лица', 'bioderma тонизирующий увлажняющий лосьон hydrabio', 'биодерма гидрабио лосьон тонизирующий увлажняющий', 'альгинатная маска janssen', 'альгинатные маски янсен', 'биодерма матирующий крем', 'биодерма себиум мат контроль', 'organic shop крем для лица', 'крем для лица органик шоп увлажняющий', 'holy land для жирной кожи', 'holy land для проблемной кожи', 'avene эмульсия', 'avene матирующая эмульсия', 'крем avene ystheal', 'avene ystheal 30 мл купить', 'крем с мочевиной для лица', 'крем для тела с мочевиной', 'крем гель алоэ', 'алоэ вера гель для лица', 'крем витамин ф', 'крем с витамином f', 'bioderma hydrabio мицеллярная вода', 'биодерма гидрабио мицеллярная вода', 'альгинатная маска algomask', 'альгинатные маски франция', 'крем vichy idealia', 'крем виши идеалия дневной', 'крем с ана кислотами', 'средство для умывания с кислотами', 'коллаген гель', 'коллагеновая мазь', 'secret key патчи купить', 'патчи секрет кей', 'патчи с золотом купить', 'патчи для глаз цена в москве', 'эмульсия урьяж', 'uriage эмульсия', 'маска gigi solar energy', 'маска gigi solar energy купить', 'бальзам для губ сиберика', 'бальзам для губ натура сиберика', 'la roche posay успокаивающий тоник', 'тоник ля рош позе для чувствительной кожи', 'крем от растяжек vichy', 'vichy от растяжек', 'bioderma sensibio ds гель', 'биодерма сенсибио дс гель очищающий', 'крем ночной для похудения', 'крем для тела chocolate slim', 'holika holika bb купить', 'бб крем холика холика купить', 'крем la roche posay lipikar', 'крем lipikar', 'крем виши от растяжек купить', 'крем от растяжек vichy купить', 'биодерма себиум сыворотка', 'биодерма себиум серум', 'la roche posay effaclar h крем', 'la roche posay effaclar h гель', 'крем с пептидами купить', 'крем для лица с пептидами купить', 'антивозрастной крем виши', 'крем виши для лица', 'биодерма мусс для умывания', 'биодерма сенсибио мусс очищающий', 'avene от купероза', 'авен от купероза', 'weleda крем для лица', 'веледа крем для лица', 'la roche posay effaclar купить', 'la roche posay effaclar h купить', 'биодерма себиум вода', 'биодерма себиум вода очищающая', 'avene cleanance купить', 'avene cleanance гель для умывания купить', 'урьяж крем с кислотами', 'урьяж крем для комбинированной кожи', 'weleda от растяжек купить', 'веледа от растяжек цена', 'крем урьяж восстанавливающий', 'крем урьяж исеак', 'uriage тональный крем', 'урьяж тональный крем', 'la roche posay для жирной кожи', 'la roche posay для проблемной кожи', 'крем авен от прыщей', 'avene от прыщей', 'биодерма крем для тела', 'биодерма молочко для тела', 'качественный крем для лица', 'крем для лица с натуральным составом', 'avene бальзам', 'бальзам для губ авен', 'биодерма гидрабио гель крем', 'bioderma hydrabio увлажняющий гель крем', 'аравия энзимный пилинг для лица', 'aravia энзимный пилинг', 'janssen ампулы', 'ампулы янсен', 'веледа крем для глаз', 'веледа крем вокруг глаз', 'крем для проблемной кожи с спф', 'крем для проблемной кожи с spf', 'librederm сыворотка активатор гиалуроновая', 'librederm активатор гиалуроновая увлажняющая сыворотка', 'альгинатная маска anskin', 'альгинатная маска анскин', 'крем виши от пигментных пятен', 'крем виши от пигментных пятен на лице', 'bioderma atoderm бальзам для губ', 'биодерма бальзам для губ', 'levrana сыворотка для лица', 'levrana сыворотка', 'крем с липосомами', 'крем для лица с липосомами', 'крем планета органика', 'planeta organica купить', 'bioderma скраб', 'биодерма скраб для лица', 'сыворотка с эффектом ботокса', 'сыворотка с эффектом ботокса для лица', 'купить крем для кожи вокруг глаз', 'купить крем вокруг глаз от морщин', 'биодерма нодэ дс', 'биодерма нодэ ds', 'авен себорегулирующая эмульсия', 'крем с куркумой для лица', 'пилинг скатка для лица купить', 'avene мицеллярный лосьон', 'бальзам для губ заживляющий', 'крем эластин коллаген кристина', 'капсулы янсен с фитоэстрогенами', 'крем вокруг глаз кристина', 'янсен крем для век', 'альгинатные маски теана', 'la roche posay мицеллярная вода купить', 'librederm гиалуроновый крем для кожи вокруг глаз', 'weleda тоник для лица', 'холи ленд сыворотка с витамином с', 'noreva exfoliac купить в москве', 'детокс скраб для тела', 'крем guam от растяжек', 'elizavecca пилинг скатка', 'avene маска для глубокого очищения', 'жидкое мыло натура сиберика', 'avene eluage крем', 'крем либридерм стволовые клетки', 'термальная вода librederm', 'librederm бальзам для губ', 'jurassic spa тональный крем', 'тоник для лица с ромашкой', 'альгинатная маска алоэ', 'крем антибактериальный для кожи', 'ля рош позе пенка для умывания', 'крем la roche posay hydreane legere', 'gigi lipacid крем', 'крем для сверхчувствительной кожи avene', 'золотая альгинатная маска для лица', 'ciracle сыворотка с витамином с', 'крем лореаль с гиалуроновой кислотой', 'крем yuza sorbet erborian', 'урьяж крем от пигментных пятен', 'крем урьяж изофиль', 'урьяж пенка для умывания', 'teana гиалуроновая кислота в ампулах', 'gigi bioderm lotion купить', 'gigi mesoactive купить', 'weleda разглаживающий увлажняющий крем уход', 'noreva aquareva купить', 'teana cb мультиламеллярный крем себоконтроль матирующий', 'альгинатная маска либридерм цена', 'gigi крем с азелаиновой кислотой', 'christina theraskin купить', 'клиник гоммаж скраб купить', 'uriage keratosane 30 купить', 'la roche posay hydreane extra riche купить', 'урьяж маска гоммаж', 'профессиональная косметика для проблемной кожи с акне', 'крем для лица лореаль роскошь питания дневной', 'secret key finish up bb cream купить', 'крем гель восстанавливающий баланс nature's', 'крем флюид для лица planeta organica', 'урьяж прурисед гель', 'крем концентрат ультраувлажнение jurassic spa', 'крем коктейль энергетик', 'кремы наборы для ухода за телом', 'teana огненный топаз пилинг для лица купить', 'интенсивный двухфазный биокомплекс для омоложения', 'holy land boldcare лосьон купить', 'крем корректор тона сс cream', 'биодерма от пигментных пятен', 'planeta organica гель мусс для умывания', 'gigi тоник азуленовый', 'крем с аминофиллином от целлюлита купить', 'крем виши с гиалуроновой кислотой', 'christina пилинг гоммаж с витамином е', 'скраб для лица планета органика', 'термальная вода ля рош позе купить', 'купить крем с ретинолом для лица', 'кислородная маска для лица купить', 'клиник гель для лица увлажняющий', 'la roche posay маска увлажняющая', 'французская натуральная косметика', 'крем геволь с мочевиной', 'крем теана мультиламеллярный крем', 'holika holika petit bb aqua купить', 'vichy пенка для умывания', 'аравия тоник детоксицирующий', 'кислотная маска кора', 'christina rose de mer тональный крем', 'крем для лица echoice с коллагеном', 'аравия с ана кислотами крем', 'sesderma azelac крем', 'mizon морской коллаген', 'фотодерм спот крем spf 50', 'крем mustela stelatopia купить', 'энзимный пилинг янсен', 'крем для тела с мерцающим эффектом купить', 'елизавета пузырьковая маска', 'скраб для лица кора', 'noreva тональный крем', 'elizavecca сыворотка гиалуроновая', 'levrana крем для лица', 'биодерма лосьон', 'увлажняющий матирующий крем для лица', 'christina увлажняющий крем', 'вв крем матирующий', 'la roche posay антивозрастной уход', 'крем ля рош позе толеран', 'weleda крем с календулой', 'альгинатные маски от прыщей', 'холодное гидрирование лица купить', 'купить крем кора с эффектом ботокса', 'крем ducray', 'уход за лицом цена', 'elizavecca сыворотка с витамином с', 'крем для лица с витамином с', 'биодерма цикабио лосьон', 'бб крем с эффектом сияния', 'крем для лица echoice с гиалуроновой кислотой', 'сыворотка для лица увлажняющая holika holika', 'сыворотка с экстрактом улитки', 'крем для тела органик', 'тканевая маска с улиткой', 'купить ducray elution', 'желтый пилинг для лица цена', 'крем толеран ультра', 'биодерма фотодерм макс spf 50', 'аравия пилинг с молочной кислотой', 'uriage cu zn крем', 'крем для тела для очень сухой кожи', 'альгинатные маски корея купить', 'сыворотка с витамином с для лица', 'planeta organica гель для душа', 'алоэ вера гель питьевой купить в москве', 'bioderma очищающая вода', 'купить израильскую косметику в москве', 'купить косметику натура сиберика', 'крем для лица планета органика', 'биодерма нодэ', 'маска с пептидами купить', 'vichy dercos aminexil купить', 'крем лиерак для стареющей кожи', 'израильский крем от пигментных пятен', 'vichy dercos купить в москве', 'цикапласт бальзам купить', 'биодерма крем для комбинированной кожи', 'крема урьяж антивозрастные', 'урьяж термальная вода 300 мл купить', 'ciracle крем', 'крем на основе ботокса', 'клинанс косметика', 'vichy ночной крем', 'ампульная косметика для лица', 'крем с койевой кислотой', 'holika holika бб крем для лица', 'крем с аминокислотами', 'крем для тела с гиалуроновой кислотой', 'крем для лица для комбинированной кожи', 'крем для глаз с spf', 'крем для век с пептидами', 'крем от растяжек для беременных купить', 'биодерма фотодерм', 'авен для атопичной кожи', 'урьяж ночной крем', 'крем виши для проблемной кожи', 'крем лифтинг для тела guam', 'ля рош позе крем от прыщей', 'vichy aqualia thermal купить', 'крем авен антиружер', 'крем для кожи вокруг глаз mizon', 'крем ля рош с гиалуроновой кислотой', 'коллагеновая сыворотка', 'vichy маска пилинг', 'vichy крем увлажняющий', 'elizavecca маска с золотом', 'сыворотка для сужения пор', 'кислородная пенка для умывания', 'planeta organica скраб для тела', 'крем гель для век кора', 'крем актив контроль', 'librederm тоник', 'балансирующий крем janssen', 'вв крем виши', 'крем лифтинг для шеи', 'крем сыворотка кора', 'крем для лица с секретом улитки', 'крем для чистки лица', 'женьшеневый крем для лица', 'скраб пилинг для лица', 'baviphat маска', 'крем для тела sea of spa', 'урьяж очищающий гель', 'avene колд крем', 'крем с экстрактом слизи улитки против морщин', 'librederm аевит гель для умывания', 'купить constant delight в москве', 'крем гоммаж', 'крем gigi ester c', 'teana сияние кожи', 'биодерма вв крем', 'урьяж молочко для тела', 'купить жидкий кератин', 'янсен тоник', 'крем козье молоко для лица', 'secret key крем для глаз', 'christina гель для умывания', 'биодерма атодерм интенсив', 'крема олеси мустаевой', 'тканевая маска с алоэ', 'тканевая маска с гиалуроновой кислотой', 'сыворотка для лица сиберика', 'мицеллярная вода купить', 'mizon сыворотка', 'крем для лица от солнца', 'start epil универсальная', 'крем лореаль купить', 'крем филлер для лица', 'крема для чувствительной кожи лица', 'сыворотка от акне', 'крем барьер', 'ампульный концентрат', 'крем с матриксилом', 'крем с коллагеном для лица', 'крем с экстрактом граната', 'заказать тональный крем', 'domix крем', 'маски holika holika', 'алоэ вера косметика', 'holy land крем для век', 'крем black pearl', 'aravia сахарная паста', 'крем заполнитель морщин', 'крем топикрем', 'крем la biosthetique', 'la roche posay от солнца', 'васильковая вода', 'vichy сыворотка', 'mizon пилинг', 'крем для глаз сиберика', 'librederm пенка для умывания', 'organic shop кокосовый рай', 'крем для глаз с ретинолом', 'маска с женьшенем для лица', 'криосыворотка от мимических морщин', 'криосыворотка от мимических морщин teana', 'кремы от морщин с гиалуроновой кислотой', 'кремы с гликолевой кислотой', 'кремы с кислотами', 'коллагеновый комплекс', 'конжаковая губка для тела', 'конжаковый спонж clarette', 'конжаковый спонж для лица купить', 'конжаковый спонж купить', 'конжаковый спонж натуральный для лица clarette', 'конжаковый спонж с бамбуковым углем', 'концентрированная сыворотка для кожи лица', 'концентрированная сыворотка для лица', 'концентрированная сыворотка лифтинг', 'концентрированная сыворотка ля рош позе', 'корейская косметика для глаз от отеков', 'корейская косметика для отбеливания лица', 'корейская косметика для сужения пор', 'корейская косметика от купероза', 'корейская косметика от купероза на лице', 'корейская косметика от расширенных пор', 'корейские маски для сужения пор', 'корейские патчи для глаз от отеков', 'корейские патчи от отеков под глазами', 'корейские пилинги для лица с кислотами', 'корейские средства от купероза', 'корейский спрей для лица', 'корейский тоник для сужения пор', 'корректирующая маска для лица', 'корректирующая сыворотка для лица', 'косметика holika holika купить', 'косметика la roche posay для проблемной кожи', 'косметика la roche posay каталог', 'косметика la roche posay купить', 'косметика natura siberica купить', 'косметика vichy купить', 'косметика виши каталог с ценами', 'косметика виши купить', 'косметика виши купить в москве', 'косметика для беременных weleda', 'косметика для лица vichy', 'косметика для лица weleda', 'косметика для лица академи', 'косметика для лица академия', 'косметика для лица с гиалуроновой кислотой', 'косметика для лица с кислотами', 'косметика для очень сухой кожи', 'косметика для очищения кожи', 'косметика для очищения кожи лица', 'косметика для сужения пор на лице', 'косметика либридерм купить', 'косметика либридерм купить в москве', 'косметика мико купить в москве', 'косметика пилинг holy land', 'косметика пилинг скатка', 'косметика против прыщей', 'косметика с алоэ', 'косметика с биозолотом', 'косметика с лифтинг эффектом', 'косметика с минералами мертвого моря', 'косметика с морскими водорослями', 'косметика с мочевиной', 'косметика с натуральным составом', 'косметика с низкомолекулярной гиалуроновой кислотой', 'косметика с фитоэстрогенами', 'косметика с фруктовыми кислотами', 'косметика с фруктовыми кислотами для лица', 'косметика с фруктовыми кислотами для проблемной кожи', 'косметические маски для лица от морщин', 'купить патчи для глаз с гиалуроновой кислотой', 'купить очищающую воду для лица', 'купить омолаживающую косметику', 'купить ночной крем виши', 'купить ночной пилинг', 'купить ночной питательный крем', 'купить натуральный крем для зрелой кожи', 'купить натуральную розовую воду', 'купить набор карбокситерапии для лица', 'купить маску для подтяжки овала лица', 'купить маску для лица с улиткой', 'купить лосьон косметический', 'купить крем холика холика', 'купить крем сиберика')
        and date >= '2019-08-16'
        and regionId in ('213', '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow')
        and position <= 10
        and isMobile = 0
    group by domain
    having count(*) >= 6
    order by count(*) desc
    limit 12"""
client = Client(host = '192.168.10.101', user = 'read_only', password = 'User_ro', settings={'max_query_size':500000})
data = client.execute(query)
client.disconnect()

In [ ]:
from backend_logic import *
offerid = '5d56807f7e76808f64a3b555'
for key in getFilterRivalsObjectByOfferId(offerid):
    print(key)

In [ ]:
all_keywords, notRivals = getKeywordsRivals(keywords, rivals, maxPos, startDate, keywordsStr, regionsStr, regions, minCountInSerm)
all_keywords

In [ ]:
minKeywordRivals = 2
keywordsToDelete = getKeywordsToRemove(all_keywords, minKeywordRivals, regions)
keywordsToDelete

In [ ]:
filename = 'Rivals ' + offerid + ' ' + 'manual' + '.xlsx'
info = [{
    'Id КП': offerid,
    'Макс. кол-во конкурентов': maxRivalsCount,
    'Макс. отклонение ИКС': sqiDiffCoef,
    'Макс. позиция для поиска конкурентов': maxPos,
    'Мин. кол-во раз, сколько сайт должен встречаться в выдаче, чтобы считаться конкурентом': minCountInSerm,
    'Мин. кол-во конкурентов для запроса': minKeywordRivals
}]
keywordsToDeleteForExcel = []
for item in keywordsToDelete:
    for key in item['keywordsToDelete']:
        keywordsToDeleteForExcel.append({
            'Регион': item['region'],
            'Ключ для удаления': key
        })
writer = pd.ExcelWriter('./downloadable_files/' + filename)
info_df = pd.DataFrame(info)
info_df.to_excel(writer, index=False, sheet_name='Метаданные', freeze_panes = (1,0))
regions_df = pd.DataFrame(regions).rename(columns={'name': 'Регион'})['Регион'].to_frame()
regions_df.to_excel(writer, index=False, sheet_name='Регионы', freeze_panes = (1,0))
rivals_df = pd.DataFrame(rivals).rename(columns={
    'domain': 'Сайт',
    'isExcluded': 'Не конкурент',
    'countInSerm': 'Встречается в выдаче',
})[['Сайт', 'Не конкурент', 'Встречается в выдаче']]
rivals_df.to_excel(writer, index=False, sheet_name='Конкуренты', freeze_panes = (1,0))
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
all_keywords_df.to_excel(writer, index=False, sheet_name='Запросы', freeze_panes = (1,0))
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
keywordsToDeleteForExcel_df.to_excel(writer, index=False, sheet_name='Запросы для удаления', freeze_panes = (1,0))
writer.save()
print(filename)

In [ ]:
keywordsToDeleteForExcel

In [ ]:
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
keywordsToDeleteForExcel_df

In [ ]:
import codecs
import json
from backend_logic import *
getOffer('5d5d6def7e76801af8637446')